# Margins for SNP
-[ ] Build / Extract the chains    
-[ ] Get undPrices   
-[ ] Integrate undPrices to the chains   

-[ ] Get the lots   
-[ ] Get margins for with qty as the lot   
-[ ] Make logic for margins from SNP rules   
-[ ] For margins with NaN, replace margins with the one derived from logic   
-[ ] Integrate margins to chains

In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Suppress Errors
util.logToFile(DATAPATH.parent.parent / 'log' / 'ztest.log')

## Get underlying contracts and prices

In [ ]:
# imports
import asyncio
from utils import get_pickle, get_mkt_prices, pickle_me, get_file_age, assemble_snp_underlyings, assemble_nse_underlyings

In [ ]:
# Get unds
UNDS_PATH = DATAPATH / 'unds.pkl'
unds = get_pickle(UNDS_PATH)

#... check file age
MIN_UND_DAYS = 1
unds_file_age = get_file_age(UNDS_PATH)
if not unds_file_age:
    unds_age_in_days = MIN_UND_DAYS
else:
    unds_age_in_days = unds_file_age.td.total_seconds()/60/60/12 

if unds_age_in_days >= MIN_UND_DAYS:
    # Assemble underlyings
    if MARKET == 'SNP':
        unds = asyncio.run(assemble_snp_underlyings(PORT))
    else:
        unds = asyncio.run(assemble_nse_underlyings(PORT))

    # pickle underlyings
    # pickle_with_age_check(unds, unds_path, 0) # No need to age-check, for fresh base build
    pickle_me(unds, UNDS_PATH)

In [ ]:
# Make df from unds
df_unds = pd.DataFrame.from_dict(unds.items())
df_unds.columns = ['symbol', 'contract']

# Get und prices
# ... check file age.
UND_PRICE_PATH = DATAPATH / 'df_und_prices.pkl'
und_price_file_age = get_file_age(UND_PRICE_PATH)
if not und_price_file_age:
    age_in_mins = 30
else:
    age_in_mins = und_price_file_age.td.total_seconds()/60

if age_in_mins >= 30:
    df_und_prices = asyncio.run(get_mkt_prices(port=PORT, contracts=df_unds.contract))
    pickle_me(df_und_prices, UND_PRICE_PATH)
else:
    df_und_prices = get_pickle(UND_PRICE_PATH)

## Build / Extract the chains

In [ ]:
# imports
import asyncio
from utils import make_chains, pickle_me

# inputs
contracts = df_unds.contract

In [ ]:
# Get chains
CHAIN_PATH = DATAPATH / 'df_chains.pkl'
chains_file_age = get_file_age(CHAIN_PATH)

if not chains_file_age:
    age_in_days = 1
else:
    age_in_days = chains_file_age.td.total_seconds()/60/60/12
    
if age_in_days >= 1:
    df_chains = asyncio.run(make_chains(port=PORT, contracts=contracts, MARKET=MARKET))
    pickle_me(df_chains, CHAIN_PATH)
else:
    df_chains = get_pickle(CHAIN_PATH)

In [ ]:
df_chains

## Integrate undPrices and undIV to the chains

In [ ]:
und_price_dict = df_und_prices.set_index('symbol').price.dropna().to_dict()
und_iv_dict = df_und_prices.set_index('symbol').iv.dropna().to_dict()

# Replace undPrice and ivs where available
df_chains.undPrice = df_chains.symbol.map(und_price_dict).fillna(df_chains.undPrice)
df_chains.iv = df_chains.symbol.map(und_iv_dict).fillna(df_chains.iv)

## Re-calculate `sigma` and `strike_sdev`

In [ ]:
from utils import compute_strike_sd_right

df_chains = compute_strike_sd_right(df_chains)

## Get chains with closest strike price to underlying

In [ ]:
# imports
from utils import get_strike_closest_to_und

In [ ]:
df_ch = get_strike_closest_to_und(df_chains)

## Get the lots

In [ ]:
from utils import get_lots

lots_dict = dict()

for k, v in unds.items():
    if v.exchange.upper() == 'NSE':
        lots_dict[k] = get_lots(v)
    else:
        lots_dict[k] = 1
        
# integrate with df_ch
df_ch['lot'] = df_ch.symbol.map(lots_dict)

## Get margins for with qty as the lot

In [ ]:
# imports
from utils import opt_margins_with_lot_check
import numpy as np


In [ ]:
# First run for margins with multiply lots
df_margins = opt_margins_with_lot_check(df_ch)

In [ ]:
# For remaining do a second run without multiply lots

from utils import join_my_df_with_another
df_rem = df_margins[df_margins.margin.isnull()]
df_remaining = join_my_df_with_another(df_ch, df_rem, ['symbol', 'expiry', 'strike', 'right'])\
                    .dropna(subset=['secType'])\
                        .reset_index()

df_r = opt_margins_with_lot_check(df_remaining, multiply_lot=False)
df_r = df_r.dropna(subset=['margin'])

cols = ['symbol', 'expiry', 'strike', 'right']
df_margins = df_margins.set_index(cols).fillna(df_r.set_index(cols), axis=1).reset_index()

## Integrate with maximum margin, wherever possible

In [ ]:
# Integration of margins and commissions

cols = ['symbol', 'expiry', 'strike', 'right']
mgn_cols = cols + ['margin', 'comm']
df_chm = pd.merge(df_ch, df_margins[mgn_cols], on=cols, suffixes=[False, '_y']).\
           drop(columns=['strk_near_und'], errors='ignore')

chain_cols = ['symbol', 'expiry']
ch_cols = chain_cols + ['lot', 'margin', 'comm']

df_out = pd.merge(df_chains, df_chm[ch_cols], on=chain_cols, suffixes=[False, '_y'])

# fill missing commissions with max per symbol
commissions = df_chm.groupby('symbol').comm.max().to_dict()
df_out.comm = df_out.comm.fillna(df_out.symbol.map(commissions))

# fill remaining commissions
df_out.comm = df_out.comm.fillna(max(commissions.values()))

# fill margins
mgn_dict = df_out.groupby('symbol').margin.max().to_dict()
cond = df_out.margin.isnull()
df_out.loc[cond, 'margin'] = df_out[cond].symbol.map(mgn_dict)

# make zero margin as nan
zero_margin_condition = df_out.margin == 0
df_out.loc[zero_margin_condition, 'margin'] = np.nan

## Make logic for margins from SNP rules

In [ ]:
df_comp = df_out[df_out.margin.isnull()]

## Replace `nan` margins with the one derived from logic